##CNN Model

Importing libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


Set up the dataset and data augmentation

In [ ]:
# Define the path to your train folder containing three sub-folders (one for each class).
train_path = 'train'

# Set up data augmentation to improve the model's performance and prevent overfitting.

datagen = ImageDataGenerator(
    rescale=1./255,       # Normalize pixel values between 0 and 1
    rotation_range=20,    # Randomly rotate images within 20 degrees
    width_shift_range=0.1,  # Randomly shift the width of images by 10%
    height_shift_range=0.1,  # Randomly shift the height of images by 10%
    shear_range=0.2,       # Shear transformation
    zoom_range=0.2,        # Randomly zoom in images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'    # Fill missing pixels with the nearest value
)

# Load and augment the images from the train directory.
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),  # Resize the images to 150x150 pixels
    batch_size=32,
    class_mode='categorical'  # For multi-class classification
)


Create the CNN model

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Dropout layer to reduce overfitting
    Dense(3, activation='softmax')  # 3 classes
])

Compile the model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Train the model

In [ ]:
epochs = 10
model.fit(train_generator, epochs=epochs)

Save the model as an .h5 file after training

In [ ]:
model.save('trained_model.h5')


Load the model and print the summary

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model('trained_model.h5')
print(loaded_model.summary())


Plot the loss and accuracy graph during training

In [ ]:
import matplotlib.pyplot as plt

# Access the history object that was returned after training the model
history = model.history

# Plot the loss and accuracy during training
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


##RCNN Models

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array


Data Preparation

In [ ]:
import torch
import torchvision
from torchvision.transforms import Compose, ToTensor, Resize, CenterCrop
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Data preparation and transformation
def load_and_resize_image(image, size=(256, 256)):
    return torchvision.transforms.functional.resize(image, size)

transform = Compose([ToTensor(), load_and_resize_image])
train_dataset = ImageFolder('train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)


# If you want to check the class names and the corresponding indices:
print(train_dataset.classes)
print(train_dataset.class_to_idx)

# Example of accessing an image and its label
for images, labels in train_loader:
    print(images.shape)  # Should be (batch_size, channels, height, width)
    print(labels)        # Tensor containing class indices (0, 1, or 2 in your case)
    break

VGG16 Model fine tuning

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision.transforms import Compose, ToTensor, Resize, CenterCrop
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data preparation and transformation
def load_and_resize_image(image, size=(256, 256)):
    image = torchvision.transforms.functional.resize(image, size)
    return image.to(device)

transform = Compose([ToTensor(), load_and_resize_image])
train_dataset = ImageFolder('train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Load the VGG-16 model
model = vgg16(pretrained=True)

# Modify the model to match the number of classes in your dataset
num_classes = len(train_dataset.classes)
model.classifier[-1] = torch.nn.Linear(4096, num_classes)

# Move the model to the GPU if available
if torch.cuda.is_available():
    model = model.to(device)
    
# Set up the optimizer and the learning rate scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Define the loss function
criterion = torch.nn.CrossEntropyLoss()

# Fine-tune the model
num_epochs = 5  # You can adjust this according to your needs

losses = []
accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_predictions
    losses.append(epoch_loss)
    accuracies.append(epoch_accuracy)

    lr_scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")
    
# Save the fine-tuned model as .h5
torch.save(model.state_dict(), 'vgg16.pth')

# Plot Loss and Accuracy Graphs
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), losses, 'b-')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), accuracies, 'r-')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')

plt.tight_layout()
plt.show()


MibleNet RCNN Fine tuning

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision.transforms import Compose, ToTensor, Resize, CenterCrop
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import mobilenet_v2

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data preparation and transformation
def load_and_resize_image(image, size=(256, 256)):
    return torchvision.transforms.functional.resize(image, size)

transform = Compose([ToTensor(), load_and_resize_image])
train_dataset = ImageFolder('train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Load the MobileNetV2 model and move it to the GPU
model = mobilenet_v2(pretrained=True).to(device)

# Modify the model to match the number of classes in your dataset
num_classes = len(train_dataset.classes)
model.classifier[1] = torch.nn.Linear(1280, num_classes).to(device)

# Set up the optimizer and the learning rate scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Define the loss function
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 10  # You can adjust this according to your needs
losses = []
accuracies = []


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_predictions
    losses.append(epoch_loss)
    accuracies.append(epoch_accuracy)

    lr_scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")


# Save the fine-tuned model as .h5
torch.save(model.state_dict(), 'mobilenet_v2.pth')

# Plot Loss and Accuracy Graphs
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), losses, 'b-')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), accuracies, 'r-')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')


plt.tight_layout()
plt.show()
